In the terminal:

ssh -i ~/.ssh/id_rsa dana@54.202.92.12

psql   
\connect baseball #connect to baseball database   
\d #show tables   
DROP TABLE master; # remove a table

In [1]:
#import sqlalchemy
from sqlalchemy import create_engine

import pandas as pd
from sshtunnel import SSHTunnelForwarder


In [2]:

AWS_IP_ADDRESS = '54.188.60.161'
AWS_USERNAME = 'dana'
SSH_KEY_PATH = '/Users/dana/.ssh/id_rsa'

server = SSHTunnelForwarder(
    AWS_IP_ADDRESS,
    ssh_username=AWS_USERNAME,
    ssh_pkey=SSH_KEY_PATH,
    remote_bind_address=('localhost', 5432),
)

server.start()
print(server.is_active, server.is_alive, server.local_bind_port)

True True 50091


In [3]:

# Postgres username, password, and database name
POSTGRES_IP_ADDRESS = 'localhost' ## This is localhost because SSH tunnel is active
POSTGRES_PORT = str(server.local_bind_port)
POSTGRES_USERNAME = 'dana'     ## CHANGE THIS TO YOUR POSTGRES USERNAME
POSTGRES_PASSWORD = 'dana'     ## CHANGE THIS TO YOUR POSTGRES PASSWORD
POSTGRES_DBNAME = 'baseball'

# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=POSTGRES_USERNAME, 
                        password=POSTGRES_PASSWORD,
                        ipaddress=POSTGRES_IP_ADDRESS,
                        port=POSTGRES_PORT,
                        dbname=POSTGRES_DBNAME))

# Create the connection
cnx = create_engine(postgres_str)

### 1. What was the total spent on salaries by each team, each year?

In [4]:
sql_query = '''SELECT teamid, yearid, SUM(salary) as total
               FROM salaries 
               GROUP BY teamid, yearid
               ORDER BY teamid;'''

pd.read_sql_query(sql_query, cnx)

,teamid,yearid,total
0,ANA,2001,47535167.0
1,ANA,2004,100534667.0
2,ANA,2003,79031667.0
3,ANA,2000,51464167.0
4,ANA,1998,41281000.0
5,ANA,2002,61721667.0
6,ANA,1997,31135472.0
7,ANA,1999,55388166.0
8,ARI,2000,81027833.0
9,ARI,2001,85082999.0


### 2. What is the first and last year played for each player?   
Hint: Create a new table from 'Fielding.csv'.



On AWS in psql, create a new table called Fielding and populate with data

```sql   
CREATE TABLE IF NOT EXISTS Fielding (   
        playerID varchar(20) NOT NULL,   
        yearID int NOT NULL,   
        stint int NOT NULL,   
        teamID text DEFAULT NULL,   
        lgID text DEFAULT NULL,
        POS text DEFAULT NULL,
        G int DEFAULT NULL,
        GS double precision DEFAULT NULL,
        InnOuts double precision DEFAULT NULL,
        PO double precision DEFAULT NULL,
        A double precision DEFAULT NULL,
        E double precision DEFAULT NULL,
        DP double precision DEFAULT NULL,
        PB double precision DEFAULT NULL,
        WP double precision DEFAULT NULL,
        SB double precision DEFAULT NULL,
        CS double precision DEFAULT NULL,
        ZR double precision DEFAULT NULL,
        PRIMARY KEY (playerID, yearID,POS,stint)
);
```

then load the data into the table

```sql
COPY Fielding FROM '/home/dana/baseballdata/Fielding.csv' DELIMITER ',' CSV HEADER;
```


In [6]:
sql_query = '''SELECT playerid, min(yearid), max(yearid)
               FROM Fielding 
               GROUP BY playerid;'''

pd.read_sql_query(sql_query, cnx)

,playerid,min,max
0,gagnied01,1914,1915
1,ramospe01,1955,1970
2,putkolu01,2012,2013
3,boyerke01,1955,1969
4,dillibo01,1946,1951
5,jeterde01,1995,2013
6,sperrst01,1936,1938
7,vidrojo01,1997,2008
8,izturce01,2001,2013
9,clarkda06,2007,2007


### 3. Who has played the most all star games?

In [7]:
sql_query = '''SELECT playerID, COUNT(yearID) AS num
               FROM AllstarFull 
               GROUP BY playerID
               ORDER BY num DESC
               LIMIT 1;'''

pd.read_sql_query(sql_query, cnx)

,playerid,num
0,aaronha01,25


### 4. Which school has generated the most distinct players? Hint: Create new table from 'CollegePlaying.csv'.

On AWS in psql, create a new table called Fielding and populate with data

```sql   
CREATE TABLE IF NOT EXISTS SchoolsPlayers (   
        playerID varchar(20) NOT NULL,   
        schoolID varchar(20) NOT NULL,   
        yearMin int NOT NULL,   
        yearMax int NOT NULL,
        PRIMARY KEY (playerID, schoolID)
);
```

then load the data into the table

```sql
COPY SchoolsPlayers FROM '/home/dana/baseballdata/SchoolsPlayers.csv' DELIMITER ',' CSV HEADER;
```


In [8]:
sql_query = '''SELECT schoolID, COUNT(playerID) AS num
               FROM SchoolsPlayers 
               GROUP BY schoolID
               ORDER BY num DESC
               LIMIT 1;'''

pd.read_sql_query(sql_query, cnx)

,schoolid,num
0,usc,102


### 5. Which players have the longest career? Assume that the debut and finalGame columns comprise the start and end, respectively, of a player's career. Hint: Create a new table from 'Master.csv'. Also note that strings can be converted to dates using the DATE function and can then be subtracted from each other yielding their difference in days.

On AWS in psql, create a new table called Master and populate with data

```sql   
CREATE TABLE IF NOT EXISTS Master (   
        playerID varchar(20) NOT NULL,   
        nameFirst varchar(20),   
        nameLast varchar(20),   
        debut varchar(20), 
        finalGame varchar(20),
        PRIMARY KEY (playerID)
);
```

then load the data into the table

```sql
COPY Master FROM PROGRAM 'cut -d "," -f 1,14,15,21,22 /home/dana/baseballdata/Master.csv' DELIMITER ',' CSV HEADER;
```

In [9]:
sql_query = '''SELECT nameFirst, nameLast, DATE(finalGame)-DATE(debut) AS diff
               FROM Master 
               WHERE debut IS NOT NULL
               GROUP BY playerID
               ORDER BY diff DESC
               LIMIT 1;'''

pd.read_sql_query(sql_query, cnx)

,namefirst,namelast,diff
0,Nick,Altrock,12862


### 6. What is the distribution of debut months? Hint: Look at the DATE and EXTRACT functions.

In [10]:
sql_query = '''SELECT EXTRACT(MONTH from DATE(debut)) as m
               FROM Master 
               WHERE debut IS NOT NULL
               GROUP BY m;'''

pd.read_sql_query(sql_query, cnx)

,m
0,8.0
1,5.0
2,9.0
3,4.0
4,10.0
5,7.0
6,6.0
7,3.0


### 7. What is the effect of table join order on mean salary for the players listed in the main (master) table? Hint: Perform two different queries, one that joins on playerID in the salary table and other that joins on the same column in the master table. You will have to use left joins for each since right joins are not currently supported with SQLalchemy.

In [11]:
sql_query = '''SELECT AVG(salaries.salary)
               FROM Master 
               LEFT JOIN salaries ON Master.playerID=salaries.playerID;'''

pd.read_sql_query(sql_query, cnx)

,avg
0,1.864357e+06


In [12]:
sql_query = '''SELECT AVG(salaries.salary)
               FROM salaries
               LEFT JOIN Master ON Master.playerID=salaries.playerID;'''

pd.read_sql_query(sql_query, cnx)

,avg
0,1.864357e+06


There is no effect on JOIN order

In [13]:
# close connection
server.close()